In [1]:
import os
import sys
import pandas
# requires 'pip install pyarrow'
import pyarrow
import pyarrow.parquet as pq

In [2]:
s="id|chunk_id,''),coalesce(coord_ra,''),coalesce(coord_decl,''),coalesce(coord_htmId20,''),coalesce(parent,''),cast(flags_badcentroid as int),coalesce(centroid_sdss_x,''),coalesce(centroid_sdss_y,''),coalesce(centroid_sdss_xVar,''),coalesce(centroid_sdss_xyCov,''),coalesce(centroid_sdss_yVar,''),cast(centroid_sdss_flags as int),cast(flags_pixel_edge as int),cast(flags_pixel_interpolated_any as int),cast(flags_pixel_interpolated_center as int),cast(flags_pixel_saturated_any as int),cast(flags_pixel_saturated_center as int),cast(flags_pixel_cr_any as int),cast(flags_pixel_cr_center as int),coalesce(centroid_naive_x,''),coalesce(centroid_naive_y,''),coalesce(centroid_naive_xVar,''),coalesce(centroid_naive_xyCov,''),coalesce(centroid_naive_yVar,''),cast(centroid_naive_flags as int),coalesce(centroid_gaussian_x,''),coalesce(centroid_gaussian_y,''),coalesce(centroid_gaussian_xVar,''),coalesce(centroid_gaussian_xyCov,''),coalesce(centroid_gaussian_yVar,''),cast(centroid_gaussian_flags as int),coalesce(shape_sdss_Ixx,''),coalesce(shape_sdss_Iyy,''),coalesce(shape_sdss_Ixy,''),coalesce(shape_sdss_IxxVar,''),coalesce(shape_sdss_IxxIyyCov,''),coalesce(shape_sdss_IxxIxyCov,''),coalesce(shape_sdss_IyyVar,''),coalesce(shape_sdss_IyyIxyCov,''),coalesce(shape_sdss_IxyVar,''),cast(shape_sdss_flags as int),coalesce(shape_sdss_centroid_x,''),coalesce(shape_sdss_centroid_y,''),coalesce(shape_sdss_centroid_xVar,''),coalesce(shape_sdss_centroid_xyCov,''),coalesce(shape_sdss_centroid_yVar,''),cast(shape_sdss_centroid_flags as int),cast(shape_sdss_flags_unweightedbad as int),cast(shape_sdss_flags_unweighted as int),cast(shape_sdss_flags_shift as int),cast(shape_sdss_flags_maxiter as int),coalesce(flux_psf,''),coalesce(flux_psf_err,''),cast(flux_psf_flags as int),coalesce(flux_psf_psffactor,''),cast(flux_psf_flags_psffactor as int),cast(flux_psf_flags_badcorr as int),coalesce(flux_naive,''),coalesce(flux_naive_err,''),cast(flux_naive_flags as int),coalesce(flux_gaussian,''),coalesce(flux_gaussian_err,''),cast(flux_gaussian_flags as int),coalesce(flux_gaussian_psffactor,''),cast(flux_gaussian_flags_psffactor as int),cast(flux_gaussian_flags_badcorr as int),coalesce(flux_sinc,''),coalesce(flux_sinc_err,''),cast(flux_sinc_flags as int),coalesce(centroid_record_x,''),coalesce(centroid_record_y,''),coalesce(classification_extendedness,''),coalesce(aperturecorrection,''),coalesce(aperturecorrection_err,''),coalesce(refFlux,''),coalesce(refFlux_err,''),coalesce(objectId,''),coalesce(coord_raVar,''),coalesce(coord_radeclCov,''),coalesce(coord_declVar,''),coalesce(exposure_id,''),coalesce(exposure_filter_id,''),coalesce(exposure_time,''),coalesce(exposure_time_mid,''),coalesce(cluster_id,''),coalesce(cluster_coord_ra,''),coalesce(cluster_coord_decl"
s=s.replace(",coalesce(","|").replace(",cast(","|").replace(" as int)","").replace("\''","").replace(",)","")
column_names=s.split("|")
print(column_names)

['id', 'chunk_id', 'coord_ra', 'coord_decl', 'coord_htmId20', 'parent', 'flags_badcentroid', 'centroid_sdss_x', 'centroid_sdss_y', 'centroid_sdss_xVar', 'centroid_sdss_xyCov', 'centroid_sdss_yVar', 'centroid_sdss_flags', 'flags_pixel_edge', 'flags_pixel_interpolated_any', 'flags_pixel_interpolated_center', 'flags_pixel_saturated_any', 'flags_pixel_saturated_center', 'flags_pixel_cr_any', 'flags_pixel_cr_center', 'centroid_naive_x', 'centroid_naive_y', 'centroid_naive_xVar', 'centroid_naive_xyCov', 'centroid_naive_yVar', 'centroid_naive_flags', 'centroid_gaussian_x', 'centroid_gaussian_y', 'centroid_gaussian_xVar', 'centroid_gaussian_xyCov', 'centroid_gaussian_yVar', 'centroid_gaussian_flags', 'shape_sdss_Ixx', 'shape_sdss_Iyy', 'shape_sdss_Ixy', 'shape_sdss_IxxVar', 'shape_sdss_IxxIyyCov', 'shape_sdss_IxxIxyCov', 'shape_sdss_IyyVar', 'shape_sdss_IyyIxyCov', 'shape_sdss_IxyVar', 'shape_sdss_flags', 'shape_sdss_centroid_x', 'shape_sdss_centroid_y', 'shape_sdss_centroid_xVar', 'shape_sdss

In [3]:
def datatype(dt):
    if dt == 'bigint':
        return 'Int64'
    elif dt == 'real':
        return 'float32'
    elif dt == 'float':
        return 'float64'
    elif dt == 'int':
        return 'Int32'
    elif dt=='bit':
        return 'bool'
    else:
        return dt
    
def source_schema(df,sch):
    for column in df.columns:
        if not(column in sch.keys()):
            print('cannot find column ',column)
            continue
        dt=datatype(sch[column])
        df[[column]]=df[[column]].astype(dt)
    return df

In [4]:
sch=pandas.read_csv('Source_schema.csv')
sch.set_index('column_name',inplace=True)
sch=sch.to_dict()['data_type']
sch['chunk_id']=sch['chunkID']


In [5]:
df=pandas.DataFrame(columns=column_names)
#df=source_schema(df,sch)

In [6]:
pandas.__version__

'0.24.0'

In [7]:
fn=f='/home/idies/workspace/Temporary/gerard/scratch/LSST/csv/Source_10030_1000.csv'
df=pandas.read_csv(fn,names=column_names)


In [8]:
df=source_schema(df,sch)
df.dtypes

id                                   Int64
chunk_id                             Int32
coord_ra                           float32
coord_decl                         float32
coord_htmId20                        Int64
parent                               Int64
flags_badcentroid                     bool
centroid_sdss_x                    float32
centroid_sdss_y                    float32
centroid_sdss_xVar                 float32
centroid_sdss_xyCov                float32
centroid_sdss_yVar                 float32
centroid_sdss_flags                   bool
flags_pixel_edge                      bool
flags_pixel_interpolated_any          bool
flags_pixel_interpolated_center       bool
flags_pixel_saturated_any             bool
flags_pixel_saturated_center          bool
flags_pixel_cr_any                    bool
flags_pixel_cr_center                 bool
centroid_naive_x                   float32
centroid_naive_y                   float32
centroid_naive_xVar                float32
centroid_na

In [17]:
v=df['parent'].values
type(v)
v.astype('int64')


ValueError: cannot convert float NaN to integer

In [9]:
t=pyarrow.Table.from_pandas(df,preserve_index=False)
pq.write_table(t,f='/home/idies/workspace/Temporary/gerard/scratch/LSST/parquet/Source_10030_1000.parquet')

ArrowTypeError: ('Did not pass numpy.dtype object', 'Conversion failed for column id with type Int64')

In [ ]:
parquet_file = pq.ParquetFile(f))

In [ ]:
d='/home/idies/workspace/Temporary/gerard/scratch/LSST/csv/'
p=d.replace("csv","parquet")
for f in os.listdir(d):
    fn=d+f
    if df is None:
        df = pandas.read_csv(fn,names=column_names)
        df= source_schema(df)
    else:
        _df=source_schema(pandas.read_csv(fn,names=column_names),sch)
        df.append(_df,ignore_index=True,sort=True)
#     t=pyarrow.Table.from_pandas(df,preserve_index=False)
#     pq.write_table(t,p+f.replace(".csv",".parquet"),)

In [ ]:
_df=pandas.read_csv(fn)
_df

In [ ]:
f='/home/idies/workspace/Temporary/gerard/scratch/LSST/parquet/Source_10030_1000.parquet'
parquet_file = pq.ParquetFile(f)

In [ ]:
print(parquet_file.metadata.schema)